In [ ]:
# pip install adafruit-circuitpython-pca9685 adafruit-circuitpython-servokit RPi.GPIO

In [ ]:
import time
from adafruit_servokit import ServoKit
class Servo:
    # adafruit_servokit.ServoKit(*, channels, i2c=None, address=64, reference_clock_speed=25000000, frequency=50) 
    kit = ServoKit(channels=16)  
    
    servo_map = { # 
        'clamp'     : {'pin':15, 'pulse_width_range':( 600,1240), 'default':   0, 'open'      :  0},  
        'clamp_roll': {'pin':13, 'pulse_width_range':( 600,2800), 'default':   0, 'servo_down':  0}, 
        'clamp_yaw' : {'pin': 7, 'pulse_width_range':( 500,2650), 'default':  90, 'straight':90, 'right'     :   0}, 
        'beam_pitch': {'pin':11, 'pulse_width_range':(1200,2850), 'default': 180, 'up4rest'   :180, 'level':60}, 
        'beam_yaw'  : {'pin': 8, 'pulse_width_range':(1600,2650), 'default':  90, 'straight':90, 'right'     :   0} 
    } 
        
    def __init__(self, servo_move_speed = 0.008):
        self.__dict__['servo_move_speed'] = servo_move_speed  
        for i, j in self.servo_map.items():
            self.kit.servo[j['pin']].set_pulse_width_range( *j['pulse_width_range'] ) 
            self.__dict__[i] = j['default'] 
        
        self.kit.servo[self.servo_map['clamp'     ]['pin']].angle=self.servo_map['clamp'     ]['default'];time.sleep(1)
        self.kit.servo[self.servo_map['clamp_roll']['pin']].angle=self.servo_map['clamp_roll']['default'];time.sleep(1)
        self.kit.servo[self.servo_map['clamp_yaw' ]['pin']].angle=self.servo_map['clamp_yaw' ]['default'];time.sleep(1)
        self.kit.servo[self.servo_map['beam_pitch']['pin']].angle=self.servo_map['beam_pitch']['default'];time.sleep(1)
        self.kit.servo[self.servo_map['beam_yaw'  ]['pin']].angle=self.servo_map['beam_yaw'  ]['default'];time.sleep(1)  
                
        for i in self.servo_map.values(): 
            self.kit.servo[i['pin']].angle = None  
    
    def __setattr__(self, name, new_value): 
        if not isinstance(new_value, int): 
            self.kit.servo[self.servo_map[name]['pin']].angle = None
            #print( f'[{name}]:Stop' )
            return
        old_value = self.__getattribute__(name)
        if new_value == old_value : 
            #print( f'[{name}]:Want new move' )
            return
        step = 1 if new_value > old_value else -1 
        for i in range(old_value, new_value, step):
            self.kit.servo[self.servo_map[name]['pin']].angle = i
            time.sleep(self.servo_move_speed)
        if name in ['clamp_roll', 'beam_yaw']: 
            self.kit.servo[self.servo_map[name]['pin']].angle = None
        self.__dict__[name] = new_value  

In [ ]:
def Homing():
    servo.beam_pitch = 180
    time.sleep(1)
    servo.clamp_roll =   0
    servo.clamp_yaw  =  90
    servo.beam_yaw   =  90
    servo.beam_pitch =  57
    servo.clamp      =   0
    servo.beam_pitch = 180

def Stop():
    servo.clamp      = None
    servo.clamp_roll = None
    servo.clamp_yaw  = None
    servo.beam_pitch = None
    servo.beam_yaw   = None

def Fetch():
    servo.clamp      =   0
    servo.beam_pitch = 57 # down 
    servo.clamp =  75 # close  
    servo.beam_pitch = 120 # raise

def Pour():
    servo.beam_pitch = 90 
    servo.clamp_roll = 180
    time.sleep(1)
    servo.clamp_roll = 0  
    
def Yaw(Y):
    if Y < -140:
        Y = -140
    elif Y > 140:
        Y = 140 
    yaw = 180-150*(140+Y)/(280) 
    servo.beam_yaw = int(yaw)

In [ ]:
class Robot:
    def __init__(self):
        self.fetch_flag  = False 
        
    def json2Action(self, js):
        if js['action'] == 'homing':
            Homing()
            Stop()
            self.fetch_flag = False
            #homing_flag = True
        elif js['action'] == 'move':
            if not self.fetch_flag:
                Fetch() 
                self.fetch_flag = True
            Yaw(js['Y'])
            if abs(js['Roll']) > 120:
                Pour()
        else:
            pass 

In [ ]:
servo = Servo()
robot = Robot()

In [ ]:
import socket, json
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(("0.0.0.0", 11220))
s.listen(0)
while True: 
    print("Standing by~")
    c, a = s.accept()
    print("New connection")
    robot.json2Action({'action': 'homing'}) 
    fo = c.makefile(mode='r')  
    try:
        while True: 
            data = fo.readline() 
            if not data: 
                print("Client lost")
                c.shutdown(socket.SHUT_RDWR) 
                c.close()
                break 
            robot.json2Action(json.loads(data)) 
            print(data.__repr__())
            '''
            {'action': 'move', 'X': -11.861002, 'Y': -102.945946, 'Z': 145.269928, 'Pitch': -24.37359, 'Roll': -0.539765, 'Yaw': -29.876957}
            {'action': 'homing'}
            '''
    except ConnectionResetError as e:
        print(f'Going to wait for new connection. Reason {e}' )